<a href="https://colab.research.google.com/github/Yvnminc/COMP5329_A2/blob/main/best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive > /dev/null
!pip install torchmetrics > /dev/null
from pydrive.auth import GoogleAuth
import torch
dev = "cuda" if torch.cuda.is_available() else "cpu"
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#get the data from the drive
def get_feature(model = "ViT-L/14@336px"):
  id = '11yXddz5j-IDcH77wkMF7dW2JctRv7YN6'
  size = "B"

  if model == 'ViT-L/14@336px':
    id = '1P6CgrgiIACjnhtHsUwdY8qjZ-E3Vx_0x'
    size = "L336"
  elif model == 'ViT-L/14':
    id = '1DG4J-YF57ZsfXTzwg5EFfkzUmUFfr82h'
    size = "L"
  elif model == 'ViT-B/32':
    id = '11yXddz5j-IDcH77wkMF7dW2JctRv7YN6'
    size = "B"

  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('clip_features.zip')

  #get the data from the drive
  id = '1b-ujWaLM_jOzlRMbXVb9T-3oEibKQW1r'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('label_onehot_tensor.pt')

  !unzip clip_features.zip > /dev/null

  test_image_features = torch.load(f"test_image_features_vit{size}.pt")
  test_text_feature = torch.load(f"test_text_feature_vit{size}.pt")
  all_image_features = torch.load(f"all_image_features_vit{size}.pt")
  all_text_feature = torch.load(f"all_text_feature_vit{size}.pt")
  label_onehot_tensor = torch.load(f"label_onehot_tensor.pt")

  return all_image_features, all_text_feature, test_image_features, test_text_feature, label_onehot_tensor

In [2]:
all_image_features, all_text_feature, test_image_features, test_text_feature, label_onehot_tensor = get_feature()

In [1]:
import torch
dev = "cuda" if torch.cuda.is_available() else "cpu"
test_image_features = torch.load("test_image_features_vitL.pt", map_location = torch.device(dev))
test_text_feature = torch.load("test_text_feature_vitL.pt", map_location = torch.device(dev))
all_image_features = torch.load("all_image_features_vitL.pt", map_location = torch.device(dev))
all_text_feature = torch.load("all_text_feature_vitL.pt", map_location = torch.device(dev))
label_onehot_tensor = torch.load("label_onehot_tensor.pt", map_location = torch.device(dev))

FileNotFoundError: ignored

In [3]:
test_image_features.shape

torch.Size([10000, 768])

In [4]:
def add_weight_decay(model, weight_decay=1e-4, skip_list=()):
    decay = []
    no_decay = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue  # frozen weights
        if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list:
            no_decay.append(param)
        else:
            decay.append(param)
    return [
        {'params': no_decay, 'weight_decay': 0.},
        {'params': decay, 'weight_decay': weight_decay}]

In [5]:
import torch
import torch.nn as nn


class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=True):
        super(AsymmetricLoss, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        # Calculating Probabilities
        
        xs_pos = x
        xs_neg = 1 - x

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic CE calculation
        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)  # pt = p if t > 0 else 1-p
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            loss *= one_sided_w

        return -loss.sum()


class AsymmetricLossOptimized(nn.Module):
    ''' Notice - optimized version, minimizes memory allocation and gpu uploading,
    favors inplace operations'''

    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        # prevent memory allocation and gpu uploading every iteration, and encourages inplace operations
        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        self.targets = y
        self.anti_targets = 1 - y

        # Calculating Probabilities
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        # Basic CE calculation
        self.loss = self.targets * torch.log(self.xs_pos.clamp(min=self.eps))
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum()

In [6]:
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from torchmetrics import F1Score
from torch import optim
from torch.cuda.amp import GradScaler, autocast

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
def Trainer(model, Data, epochs, epoch_step_1, epoch_step_2, lr = 1e-3):
    torch.manual_seed(5329)
    train_data = DataLoader(TensorDataset(Data, label_onehot_tensor), batch_size=30000, shuffle = True)
    model = model.to(dev)
 
    weight_decay = 1e-4
    criterion = AsymmetricLoss(gamma_neg=0, gamma_pos=0, clip=0, disable_torch_grad_focal_loss=True)
    parameters = add_weight_decay(model, weight_decay)
    opti = optim.Adam(params=parameters, lr=lr, weight_decay=0)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(opti, milestones=[epoch_step_1,epoch_step_2], gamma = 0.1)

    epoch = epochs
    loss_list = []
    f1_list = []
    scaler = GradScaler()

    for epoch in tqdm(range(epoch), colour = 'GREEN'):
        for data, label in train_data:   
            data, label = data.to(dev), label.to(dev)

            with autocast():  # mixed precision
                output = model(data).float() 

            loss = criterion(output, label)
            model.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(opti)
            scaler.update()
            scheduler.step()
        loss_list.append(loss)
        
        if epoch % 10 == 0:
            print('Train Epoch: {}\tLoss: {:.6f}'.format(
                epoch, loss.item()))
    
    return model, loss_list

In [26]:
import torch.nn as nn
import torch.nn.functional as F

class SpatialGatingUnit(nn.Module):
    def __init__(self, d_ffn):
        super().__init__()
        self.layer_norm = nn.LayerNorm(d_ffn)
        self.fc = nn.Linear(d_ffn, d_ffn)
        
    def forward(self, x):
        u = self.layer_norm(x)
        g = torch.sigmoid(self.fc(u))
        return x * g

class gMLPBlock(nn.Module):
    def __init__(self, d_model, d_ffn):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ffn)
        self.bn1 = nn.BatchNorm1d(d_ffn)
        self.sgu = SpatialGatingUnit(d_ffn)
        self.fc2 = nn.Linear(d_ffn, d_model)

        # He initialization
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')

    def forward(self, x):
        residual = x
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.gelu(x)
        x = self.sgu(x)
        x = self.fc2(x)
        return x + residual


class FEATURE_EXTRACTOR(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 18)
        self.dropout = nn.Dropout(p = 0.6)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = self.dropout(tensor)
        tensor = F.gelu(self.fc2(tensor))
        tensor = self.dropout(tensor)
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

class DECISION_MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(18, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

In [58]:
import torch.nn as nn
import torch.nn.functional as F

class SpatialGatingUnit(nn.Module):
    def __init__(self, d_ffn):
        super().__init__()
        self.layer_norm = nn.LayerNorm(d_ffn)
        self.fc = nn.Linear(d_ffn, d_ffn)
        
    def forward(self, x):
        u = self.layer_norm(x)
        g = torch.sigmoid(self.fc(u))
        return x * g

class gMLPBlock(nn.Module):
    def __init__(self, d_model, d_ffn):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ffn)
        self.bn1 = nn.BatchNorm1d(d_ffn)
        self.sgu = SpatialGatingUnit(d_ffn)
        self.fc2 = nn.Linear(d_ffn, d_model)

        # He initialization
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')

    def forward(self, x):
        residual = x
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.gelu(x)
        x = self.sgu(x)
        x = self.fc2(x)
        return x + residual


class FEATURE_EXTRACTOR(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = gMLPBlock(768, 768)
        self.fc1 = nn.Linear(768, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 18)
        self.dropout = nn.Dropout(p = 0.6)

    def forward(self, inputs):
        tensor = self.block1(inputs)
        tensor = F.gelu(self.fc1(tensor))
        tensor = self.dropout(tensor)
        tensor = F.gelu(self.fc2(tensor))
        tensor = self.dropout(tensor)
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

class DECISION_MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(18, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpatialGatingUnit(nn.Module):
    def __init__(self, d_ffn):
        super().__init__()
        self.layer_norm = nn.LayerNorm(d_ffn)
        self.fc = nn.Linear(d_ffn, d_ffn)
        
    def forward(self, x):
        u = self.layer_norm(x)
        g = torch.sigmoid(self.fc(u))
        return x * g

class gMLPBlock(nn.Module):
    def __init__(self, d_model, d_ffn):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ffn)
        self.bn1 = nn.BatchNorm1d(d_ffn)
        self.sgu = SpatialGatingUnit(d_ffn)
        self.fc2 = nn.Linear(d_ffn, d_model)

        # He initialization
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')

    def forward(self, x):
        residual = x
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.gelu(x)
        x = self.sgu(x)
        x = self.fc2(x)
        return x + residual

class gMLP_feature(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = gMLPBlock(1536, 1536)
        self.block2 = gMLPBlock(1536, 1536)
        self.fc = nn.Linear(1536, 18)

    def forward(self, inputs):
        tensor = self.block1(inputs)
        tensor = self.block2(tensor)
        tensor = torch.sigmoid(self.fc(tensor))
        return tensor

class gMLP_decision(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = gMLPBlock(18, 18)
        self.block2 = gMLPBlock(18, 18)
        self.fc = nn.Linear(18, 18)

    def forward(self, inputs):
        tensor = self.block1(inputs)
        tensor = self.block2(tensor)
        tensor = torch.sigmoid(self.fc(tensor))
        return tensor


In [46]:
all_image_features.shape

torch.Size([30000, 768])

In [48]:
all_text_feature.shape

torch.Size([30000, 768])

In [51]:
all_feature = torch.cat((all_image_features, all_text_feature), dim = 1)
all_feature.shape

torch.Size([30000, 1536])

In [52]:
test = torch.cat((test_image_features, test_text_feature), dim = 1)
test.shape

torch.Size([10000, 1536])

In [60]:
Net, loss_list = Trainer(FEATURE_EXTRACTOR(), all_image_features, 400, 350, 400)
Net.eval()
with torch.autograd.no_grad():
    img_train = Net(all_image_features.to(dev))
    img_test = Net(test_image_features.to(dev))

  0%|          | 0/400 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 1/400 [00:00<03:05,  2.15it/s]

Train Epoch: 0	Loss: 375067.500000


  3%|▎         | 11/400 [00:04<02:47,  2.32it/s]

Train Epoch: 10	Loss: 375091.593750


  5%|▌         | 21/400 [00:09<02:44,  2.31it/s]

...

 95%|█████████▌| 381/400 [02:58<00:07,  2.41it/s]

Train Epoch: 380	Loss: 333.303864


 98%|█████████▊| 391/400 [03:02<00:03,  2.34it/s]

Train Epoch: 390	Loss: 319.794128


100%|██████████| 400/400 [03:07<00:00,  2.13it/s]


In [61]:
Net, loss_list = Trainer(FEATURE_EXTRACTOR(), all_text_feature, 400, 350, 400)
Net.eval()
with torch.autograd.no_grad():
    img_train = Net(all_image_features.to(dev))
    img_test = Net(test_image_features.to(dev))

  0%|          | 1/400 [00:00<02:44,  2.43it/s]

Train Epoch: 0	Loss: 372142.812500


  3%|▎         | 11/400 [00:06<03:11,  2.03it/s]

Train Epoch: 10	Loss: 372100.250000


  5%|▌         | 21/400 [00:10<02:39,  2.37it/s]

...

 95%|█████████▌| 381/400 [02:53<00:08,  2.30it/s]

Train Epoch: 380	Loss: 1237.836914


 98%|█████████▊| 391/400 [02:58<00:03,  2.26it/s]

Train Epoch: 390	Loss: 1264.764648


100%|██████████| 400/400 [03:02<00:00,  2.19it/s]


In [ ]:
sum_train = img_train+txt_train
sum_test = img_test+txt_test
Net, loss_list = Trainer(DECISION_MODEL(), sum_train, 1200, 800, 1000)
Net.eval()
with torch.autograd.no_grad():
    final = Net(sum_test)

  0%|          | 1/1200 [00:00<07:02,  2.84it/s]

Train Epoch: 0	Loss: 373638.312500


  1%|          | 11/1200 [00:04<07:03,  2.81it/s]

Train Epoch: 10	Loss: 373638.281250


  2%|▏         | 21/1200 [00:08<09:43,  2.02it/s]

...

 81%|████████  | 971/1200 [06:33<01:48,  2.11it/s]

Train Epoch: 970	Loss: 11616.472656


 82%|████████▏ | 981/1200 [06:37<01:28,  2.47it/s]

Train Epoch: 980	Loss: 11614.203125


 82%|████████▏ | 987/1200 [06:39<01:18,  2.72it/s]

In [33]:
Net, loss_list = Trainer(gMLP_feature(), all_feature, 300, 200, 250)
Net.eval()
with torch.autograd.no_grad():
    img_train = Net(all_image_features.to(dev))
    img_test = Net(test_image_features.to(dev))

  0%|          | 0/300 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 1/300 [00:00<02:02,  2.44it/s]

Train Epoch: 0	Loss: 401288.937500


  4%|▎         | 11/300 [00:05<02:07,  2.27it/s]

Train Epoch: 10	Loss: 401288.812500


  7%|▋         | 21/300 [00:09<02:06,  2.20it/s]

...

 94%|█████████▎| 281/300 [02:09<00:08,  2.27it/s]

Train Epoch: 280	Loss: 90.524445


 97%|█████████▋| 291/300 [02:13<00:03,  2.35it/s]

Train Epoch: 290	Loss: 90.402260


100%|██████████| 300/300 [02:17<00:00,  2.18it/s]


In [42]:
Net, loss_list= Trainer(gMLP_feature(), all_text_feature, 600, 400, 500)
Net.eval()
with torch.autograd.no_grad():
    txt_train = Net(all_text_feature.to(dev))
    txt_test = Net(test_text_feature.to(dev))

  0%|          | 0/600 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 1/600 [00:00<04:19,  2.31it/s]

Train Epoch: 0	Loss: 423676.937500


  2%|▏         | 11/600 [00:04<04:21,  2.26it/s]

Train Epoch: 10	Loss: 423676.968750


  4%|▎         | 21/600 [00:09<05:04,  1.90it/s]

...

 97%|█████████▋| 581/600 [04:23<00:08,  2.34it/s]

Train Epoch: 580	Loss: 671.401123


 98%|█████████▊| 591/600 [04:28<00:04,  1.98it/s]

Train Epoch: 590	Loss: 671.370361


100%|██████████| 600/600 [04:32<00:00,  2.20it/s]


In [43]:
sum_train = img_train+txt_train
sum_test = img_test+txt_test
Net, loss_list = Trainer(gMLP_decision(), sum_train, 1200, 800, 1000)
Net.eval()
with torch.autograd.no_grad():
    final = Net(sum_test)

  0%|          | 1/1200 [00:00<07:10,  2.79it/s]

Train Epoch: 0	Loss: 402181.875000


  1%|          | 11/1200 [00:04<07:21,  2.69it/s]

Train Epoch: 10	Loss: 402181.937500


  2%|▏         | 21/1200 [00:07<07:30,  2.62it/s]

...

 98%|█████████▊| 1181/1200 [07:45<00:07,  2.58it/s]

Train Epoch: 1180	Loss: 2483.550293


 99%|█████████▉| 1191/1200 [07:49<00:03,  2.66it/s]

Train Epoch: 1190	Loss: 2482.453613


100%|██████████| 1200/1200 [07:52<00:00,  2.54it/s]


In [56]:
import pandas as pd
y_proba = final.cpu().numpy()

resl = []
for i in y_proba:
    a = [x+1 for x in range(len(i)) if i[x] > 0.5]
    for j in range(len(a)):
        if a[j] >=12:
            a[j] = a[j]+1
    resl.append(a)
test_pred = []
for lis in resl:
    a = [str(i) for i in lis]
    test_pred.append(" ".join(a))

# make a csv file
df = pd.DataFrame(columns=["ImageID", "Labels"])

# Creating the Second Dataframe using dictionary
for index, value in enumerate(test_pred):
    df_temp = pd.DataFrame({"ImageID":"{}.jpg".format(30000+index), "Labels":" ".join([str(i) for i in [value]])}, index=[0])
    # for appending df_temp at the end of df
    df = df.append(df_temp, ignore_index = True)

df.to_csv("Predicted_labels.csv", index = False)

Streaming output truncated to the last 5000 lines.
<ipython-input-56-108931f1cf20>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp, ignore_index = True)
<ipython-input-56-108931f1cf20>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp, ignore_index = True)
<ipython-input-56-108931f1cf20>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp, ignore_index = True)
<ipython-input-56-108931f1cf20>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp, ignore_index = True)
<ipython-input-56-108931f1cf20>:23: FutureWarning: The frame.append method is depreca

In [57]:
df

,ImageID,Labels
0,30000.jpg,1
1,30001.jpg,1 3
2,30002.jpg,1
3,30003.jpg,1
4,30004.jpg,1
...,...,...
9995,39995.jpg,1
9996,39996.jpg,3 4
9997,39997.jpg,1
9998,39998.jpg,1
